## Load data

In [1]:
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd

data = pd.read_csv(r"extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Training: 
Assessable data are 528101 cases and 1015074 CBCs
Control data are 527038 cases and 1013548 CBCs
Sepsis data are 1488 cases and 1526 CBCs
$$$$$$$$$$$$$$$$$$$$
Testing: 


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 365794, Sepsis: 490
Assessable data are 180494 cases and 366284 CBCs
Control data are 180157 cases and 365794 CBCs
Sepsis data are 472 cases and 490 CBCs


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 437629, Sepsis: 448
Assessable data are 157922 cases and 438077 CBCs
Control data are 180157 cases and 437629 CBCs
Sepsis data are 438 cases and 448 CBCs


In [2]:
import torch

y_train = torch.tensor(data_analysis.get_y_train(), dtype=torch.long)
X_train = torch.tensor(data_analysis.get_X_train(), dtype=torch.float)

y_test = torch.tensor(data_analysis.get_y_test(), dtype=torch.long)
X_test = torch.tensor(data_analysis.get_X_test(), dtype=torch.float)

y_gw_test = torch.tensor(data_analysis.get_y_gw(), dtype=torch.long)
X_gw_test = torch.tensor(data_analysis.get_X_gw(), dtype=torch.float)

## Normalize data

In [3]:
def normalize(tensor):
    mean = torch.mean(tensor, dim = 0)
    std = torch.std(tensor, dim = 0)
    mean_diff = tensor - mean
    return mean_diff / std

X_train = normalize(X_train)
X_test = normalize(X_test)
X_gw_test = normalize(X_gw_test)

## Train/Validation split

In [4]:
def true_indices_like(tensor):
    return torch.ones((tensor.shape[0])).type(torch.bool)

def false_indices_like(tensor):
    return torch.zeros((tensor.shape[0])).type(torch.bool)

def split(train_features):
    tensor = true_indices_like(train_features)
    max_index = round(tensor.shape[0] * 0.8)
    train = torch.zeros(tensor.shape[0])
    train[:max_index] = 1
    
    val = torch.zeros(tensor.shape[0])
    val[max_index:] = 1
    return{
        "train": train.type(torch.bool),
        "val":val.type(torch.bool)
    }
X_train_clone = torch.clone(X_train)
y_train_clone = torch.clone(y_train)
train_data = split(X_train_clone)

train_mask = train_data["train"]
val_mask = train_data["val"]

X_train = X_train_clone[train_mask]
X_val = X_train_clone[val_mask]
y_train = y_train_clone[train_mask]
y_val = y_train_clone[val_mask]

## Define model

In [5]:
import torch.nn.functional as F
from torch.nn import Linear
import torch
from dataAnalysis.Constants import FEATURES

class NeuralNetwork(torch.nn.Module):

    def __init__(self, hidden_dim = 128, out_channels = 1):
        super(NeuralNetwork, self).__init__()
        
        input_dim = len(FEATURES)
        self.lin_0 = Linear(input_dim, hidden_dim)
        self.lin_1 = Linear(input_dim, hidden_dim)
        self.lin_end_0 = Linear(hidden_dim, out_channels)
        self.lin_end_1 = Linear(hidden_dim, out_channels)
        self.batchnorm_1 = torch.nn.BatchNorm1d(hidden_dim)

    def forward(self, x):
        x = self.lin_0(x) + self.lin_1(x)
        x = F.normalize(x, p=2., dim=-1)
        x = torch.relu(x)
        x= self.batchnorm_1(x)
        x = self.lin_end_0(x) + self.lin_end_1(x) 
        return x
            
    def predict_proba(self, x):
        with torch.inference_mode():
            self.eval()
            logits = self.forward(x)
            scores = torch.sigmoid(torch.squeeze(logits))
            scores = torch.unsqueeze(scores, 0)
            proba_predict = torch.concat((1- scores, scores), dim = 0)
            return torch.transpose(proba_predict, 0, 1)
            
    def predict(self, x):
        return torch.round(self.predict_proba(x)[:, 1])

## Shift data to device

In [6]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

X_train = X_train.to(device)
y_train = y_train.to(device)

X_val = X_val.to(device)
y_val = y_val.to(device)

X_test = X_test.to(device)
y_test = y_test.to(device)

X_gw_test = X_gw_test.to(device)
y_gw_test = y_gw_test.to(device)

WEIGHT = torch.tensor([664])
WEIGHT = WEIGHT.to(device)

print("Data shifted to the device " + str(device))

Data shifted to the device cuda:0


## Model-Wrapper class

In [7]:
import torch 

class ModelWrapper():
    def __init__(self):
        self.LEARNING_RATE = 3e-4
        self.MAX_EPOCHS = 10000

        self.model = NeuralNetwork(hidden_dim = 128, out_channels=1) 
        self.model = self.model.to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.LEARNING_RATE,betas=(0.9, 0.999), eps=1e-08)
        
        self.last_loss = 0
        self.increased_loss = 0
        self.BREAKING_THRESHOLD = 5      
    
    def validate(self):
        with torch.inference_mode():
            self.model.eval()
            out = self.model(X_val)
            loss = F.binary_cross_entropy_with_logits(torch.squeeze(out), y_val.type(torch.float32),
                                                      pos_weight=WEIGHT)
#             print(loss)
            if loss.item() > self.last_loss:
                self.increased_loss += 1
            else:
                self.increased_loss = 0
            self.last_loss = loss.item()

    def train(self):
        for epoch in range(self.MAX_EPOCHS):
#             print(epoch)
            self.model.train()
            self.optimizer.zero_grad()
            out = self.model(X_train)
            loss = F.binary_cross_entropy_with_logits(torch.squeeze(out), y_train.type(torch.float32),
                                                      pos_weight=WEIGHT)
            loss.backward()
            self.optimizer.step()
            self.validate() 

            if self.increased_loss >= self.BREAKING_THRESHOLD:
                print(f"Breaked at {str(epoch)}")
                break
            
    def get_model(self):
        return self.model    

In [8]:
modelWrapper = ModelWrapper()
modelWrapper.train()

Breaked at 434


In [9]:
model = modelWrapper.get_model()

## Save model

In [12]:
torch.save(model.state_dict(), "neural_network_state_dict.pt")

## Shift data and model back to CPU for evaluation

In [9]:
y_test = y_test.cpu()
X_test = X_test.cpu()

y_gw_test = y_gw_test.cpu()
X_gw_test = X_gw_test.cpu()
# model = model.cpu()

## Evaluation

In [10]:
from dataAnalysis.Metrics import Evaluation

evaluation = Evaluation(y_test, y_gw_test, X_test, X_gw_test)
# evaluation.plot_confusion_matrix(model)
# evaluation.get_df_metrics(model)

/home/dwalke/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


## Error evaluation

In [15]:
import time

dfs = []
times = []
for i in range(100):
    print(20*"%")
#     start = time.time()
    modelWrapper = ModelWrapper()
    modelWrapper.train()
#     print(time.time() - start)
#     times.append(time.time() - start)
    
    model = modelWrapper.get_model().cpu()
#     print(evaluation.get_df_metrics(model))
    dfs.append(evaluation.get_df_metrics(model))

%%%%%%%%%%%%%%%%%%%%
Breaked at 727
%%%%%%%%%%%%%%%%%%%%
Breaked at 588
%%%%%%%%%%%%%%%%%%%%
Breaked at 608
%%%%%%%%%%%%%%%%%%%%
Breaked at 677
%%%%%%%%%%%%%%%%%%%%
Breaked at 493
%%%%%%%%%%%%%%%%%%%%
Breaked at 446
%%%%%%%%%%%%%%%%%%%%
Breaked at 831
%%%%%%%%%%%%%%%%%%%%
Breaked at 399
%%%%%%%%%%%%%%%%%%%%
Breaked at 426
%%%%%%%%%%%%%%%%%%%%
Breaked at 498
%%%%%%%%%%%%%%%%%%%%
Breaked at 601
%%%%%%%%%%%%%%%%%%%%
Breaked at 739
%%%%%%%%%%%%%%%%%%%%
Breaked at 715
%%%%%%%%%%%%%%%%%%%%
Breaked at 729
%%%%%%%%%%%%%%%%%%%%
Breaked at 555
%%%%%%%%%%%%%%%%%%%%
Breaked at 604
%%%%%%%%%%%%%%%%%%%%
Breaked at 725
%%%%%%%%%%%%%%%%%%%%
Breaked at 656
%%%%%%%%%%%%%%%%%%%%
Breaked at 737
%%%%%%%%%%%%%%%%%%%%
Breaked at 515
%%%%%%%%%%%%%%%%%%%%
Breaked at 668
%%%%%%%%%%%%%%%%%%%%
Breaked at 546
%%%%%%%%%%%%%%%%%%%%
Breaked at 629
%%%%%%%%%%%%%%%%%%%%
Breaked at 695
%%%%%%%%%%%%%%%%%%%%
Breaked at 546
%%%%%%%%%%%%%%%%%%%%
Breaked at 487
%%%%%%%%%%%%%%%%%%%%
Breaked at 672
%%%%%%%%%%%%%%%%%%%%
Breaked

In [16]:
for df in dfs:
    print(df)

         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.051598  0.797054  0.448502   0.010068  0.874995  0.017790
1  Greifswald  0.038453  0.809581  0.450882   0.007070  0.813153  0.006566
         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.051036  0.785827  0.444837   0.009745  0.879036  0.018557
1  Greifswald  0.037784  0.793479  0.445750   0.006741  0.810296  0.006594
         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.054276  0.803330  0.450708   0.010603  0.881236  0.017667
1  Greifswald  0.039469  0.814747  0.452566   0.007290  0.813094  0.006532
         Name       MCC  F1-Micro  F1-Macro  F1-Binary    AUROC     AUPRC
0     Leipzig  0.051163  0.794689  0.447711   0.009953  0.87974  0.017213
1  Greifswald  0.039200  0.807456  0.450262   0.007133  0.81774  0.006615
         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.052866  0.7

         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.052116  0.798842  0.449114   0.010183  0.878059  0.018053
1  Greifswald  0.038493  0.812106  0.451676   0.007117  0.813850  0.006662
         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.052532  0.795140  0.447952   0.010157  0.883626  0.018572
1  Greifswald  0.037659  0.804484  0.449228   0.006887  0.812721  0.006272
         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.049793  0.781760  0.443431   0.009491  0.874010  0.014936
1  Greifswald  0.038860  0.795992  0.446616   0.006912  0.805483  0.005839
         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.051868  0.792522  0.447061   0.010005  0.880915  0.019860
1  Greifswald  0.037726  0.804922  0.449370   0.006903  0.816974  0.007228
         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.052367  

In [9]:
for t in times:
    print(t)

23.67472767829895
21.573588609695435
20.657383680343628
21.805586338043213
14.168943166732788
17.08205246925354
19.81443452835083
14.167516469955444
19.297861576080322
16.865254402160645
24.719831943511963
20.95818042755127
20.21818208694458
23.429773330688477
21.512087106704712
18.857051134109497
17.23066544532776
23.98099374771118
17.562803268432617
17.375792741775513
19.518230438232422
15.718163013458252
23.65390658378601
18.5971577167511
21.43760585784912
16.752075672149658
26.530201196670532
16.751776218414307
17.858837842941284
18.299582719802856
19.183961629867554
16.234987497329712
20.699447870254517
21.585558891296387
27.785765886306763
22.212035179138184
16.67612075805664
23.539618492126465
26.52658176422119
21.21462321281433
19.78019118309021
11.623957395553589
23.57751226425171
14.094248533248901
21.586645364761353
16.606101274490356
16.677392721176147
21.768888473510742
20.774094820022583
19.627937078475952
17.6001558303833
17.158462047576904
17.930248975753784
18.00478982

In [103]:
for df in dfs:
    print(df)

         NAME       MCC  F1-Macro     AUROC     AUPRC
0     Leipzig  0.053656  0.447750  0.885764  0.015580
1  Greifswald  0.039041  0.449935  0.817885  0.006255
         NAME       MCC  F1-Macro     AUROC    AUPRC
0     Leipzig  0.052468  0.444576  0.882996  0.01651
1  Greifswald  0.039676  0.447266  0.818936  0.00609
         NAME       MCC  F1-Macro     AUROC     AUPRC
0     Leipzig  0.051957  0.447225  0.880128  0.017834
1  Greifswald  0.038441  0.451215  0.814695  0.006421
         NAME       MCC  F1-Macro     AUROC    AUPRC
0     Leipzig  0.051493  0.446364  0.882117  0.01865
1  Greifswald  0.037578  0.448316  0.817028  0.00664
         NAME       MCC  F1-Macro     AUROC     AUPRC
0     Leipzig  0.050606  0.445684  0.878325  0.016797
1  Greifswald  0.038485  0.448404  0.810973  0.005983
         NAME       MCC  F1-Macro     AUROC     AUPRC
0     Leipzig  0.051770  0.448814  0.878872  0.018400
1  Greifswald  0.037534  0.451517  0.813145  0.007105
         NAME       MCC  F1-Macro 